<a href="https://colab.research.google.com/github/Yoshiwa1337/AISOC24/blob/main/Copy_of_IET_AI_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Operation Hacktivist

Greetings, hacktivists. This is your mission controller speaking. I'm here to give you a brief overview of the missions you will be undertaking to stop the criminal mastermind and his evil schemes. You are not just hackers. You are hacktivists. You use your skills and passion to fight for a better world. But the world is in danger.


A criminal mastermind Iv@N_T3rr1bl3 is plotting to unleash chaos and destruction with his evil schemes. He has a global network of accomplices, a vast fortune of stolen money, and a secret weapon: advanced artificial intelligence. You are the only ones who can stop him. You have three missions to complete, each one more challenging and rewarding than the last. Are you ready to hack for good?

##Mission 1: Operation Safari

Your first mission is to stop the animal trafficking. The criminal mastermind is using image recognition technology to capture and smuggle endangered animals across borders. You need to design and train an image recognition AI that can detect and expose his animal trafficking attempts. Complete this mission and you will gain up to 30 credit points which will count towards your victory at the grand hacktivist finale.

### The mission objective:

 - Build an AI model to track Iv@N_T3rr1bl3's animal trafficking network.
 - Your model should be able to iddentify and differentiate between exotic animals and common household animals.
 - You will not be provided with any datasets for this mission. It is your mission to find and label your own datasets
 - If you used/downloaded open datasets from the internet, please include the source in your reference.
 - The test data will not be publicised, but a sample test dataset containing 20 images in each categories will be released.



### AI training template

- A template is provided below with comments. Please ensure that your model output follows the exact same format.
- The result of your model will be scored by an automated scoring system. Thus, if your model output is not following the template, it might be disqualified.

In [ ]:
## Standard libraries. You are not allowed to add additional libraries

import tensorflow as tf #main tensorflow library
import numpy as np #for math
import matplotlib.pyplot as plt #for graphs
import pathlib # for directories
import PIL # for image manipulation

In [ ]:
## Pipeline for Google drive uploads
#
# As files are deleted from Colab after every session,
# it is recommended to upload your training data to Google Drive
#
# The following libraries enables you to import files from Google Drive


!pip install -U -q PyDrive #Run this once to install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# You will need to authenticate google drive. If you choose not to,
# you can manuallly upload to the colab files by clicking on the file
# icon on the left side of the panel and upload it manually.

# You can skip these codes if you chose to upload manually.


In [ ]:
# Accessing files from Google Drive
# Skip if you chose to upload manually

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Replace "your_data.zip" with the filename of the zip file you uploaded to Google Drive.
# Make sure to upload the file to the Root directory of your Google Drive (i.e. Don't put the files in another folder)

fid = drive.ListFile({'q':"title='your_data.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('your_data.zip')

In [ ]:
# UNZIP
# Once the zip file is successfully downloaded to your Colab workspace,
# Use the following command to unzip it.

# If you chose to upload the zip file manually instead of using Google Drive,
# you may use the command to unzip your files too.

!unzip your_data.zip

### Preparing the files in the directory

Your training data should have training data for each classes stored in a folder with the class name.

your_data/exotic --- Should contain images of exotic animals which would be trafficked by Iv@N_T3rr1bl3. These are the animals that you should detect.

your_data/common --- These are common household animals that should be ignored.


In [ ]:
# Rename <your_data> to the name of your folder.
data_dir = '/content/your_data'

#Make the directory a Path object
data_dir = pathlib.Path(data_dir)

#Use the Path glob function to find pathname that matches ".jpg". Use this to get the number of image inside the folder.
image_count = len(list(data_dir.glob('*/*.jpg')))

#Print the number of images in your data
print(image_count)

In [ ]:
# Setting the path variables for each of the categories
exotic = list(data_dir.glob('exotic/*'))
common = list(data_dir.glob('common/*'))

#Take a peek at the images
PIL.Image.open(str(exotic[1]))

### AI model template

Feel free to modify the codes in this section to tweak the various model parameters and training parameters.

In [ ]:
# Set a standard for the image to be used in the training.
# All images will be resized to this in order to ensure uniformity

#Important: During the automated testing, you will be supplied with images sized at 180 px X 180 px. You might want to take this into consideration

batch_size = 32
img_height = 180
img_width = 180

In [ ]:
#Set up the dataset - In this example, the validation split is set to 0.2 (20% Validation split). Feel free to tweak this number based on your dataset.

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
#Get class names
class_names = train_ds.class_names
print(class_names)

In [ ]:
#Building the model

#In this case, we are classifying into 2 classes
num_classes = 2



# Feel free to tweak the struture of the model here
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(24, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

#Compile the model
# Feel free to change the parameters here to optimize your model

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


### Train the Model

In the next Section, you will train the model. You will need to repeat this step in order to get the best performing model.

In [ ]:
#Train model

epochs=5 # Change this to tweak the number of training iteration
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

### Sample code for testing the model manually

In [ ]:
# Load an image

image_path = input("Enter the path to your uploaded image")

img = tf.keras.utils.load_img(
    image_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])




print(
    "This is most likely a {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
# See the image for yourself
PIL.Image.open(image_path)

### Downloading and submitting your model file

When you are satisfied with your model, save your model using the following code:

In [ ]:
# You can replace "MyModel" with any name
tf.saved_model.save(model, "MyModel")

# Zip the model file
!zip -r /content/MyModel.zip /content/MyModel\

# You can then download the model by right clicking on it from the Files tab on the left.

#Mission 2: Operation Cardshark

Your second mission is to track the credit card scam. The criminal mastermind is using fake websites, phishing emails, and malware to steal credit card information from online shoppers. You need to design a fraud detection system that can flag and alert any fraudulent transactions.

#Mission 3: Operation Stegano-saurus

Your third and final mission is to foil the covert communication. The criminal mastermind is using image steganography to hide secret messages in pictures that he posts on social media or sends via email. You need to design an AI that can extract and decode any hidden data within images.